In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load libraries

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
device_name

''

In [3]:
!pip install transformers
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoConfig,AutoTokenizer
import re
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import plot_confusion_matrix
import numpy as np

pd.set_option('display.max_colwidth', None)


     |████████████████████████████████| 2.1MB 6.1MB/s 
     |████████████████████████████████| 3.3MB 21.0MB/s 
     |████████████████████████████████| 901kB 35.4MB/s 


#Define pre_trained models & varaibles

In [4]:
MODEL_NAME = 'Hate-speech-CNERG/dehatebert-mono-arabic'
BATCH_SIZE = 7
N_EPOCHS = 3

#Load Dataset

In [5]:
# subm_path = '/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/sample_submission.xlsx'
df_train = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/train.xlsx')
df_valid = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/dev.xlsx')
df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/testLabels.xlsx')
# s = pd.Series([1, 2, 3, 4])

# df_test_labels = df_test_labels.set_index('id')



# label_cols = ['HS_Labels']
# print(y_test.uniqe())
# label_cols= label_cols.apply(data_classes.index)

# print (label_cols)
# df_train.tail()
# df_test_labels()

# label_cols

#Preprocessing

In [6]:
#Rename Columns
df_train = df_train.rename(columns={'off': 'OFF_Labels','hs': 'HS_Labels'})
df_valid = df_valid.rename(columns={'off': 'OFF_Labels','hs': 'HS_Labels'})
df_test_labels=df_test_labels.rename(columns={'label':'HS_Labels'})

In [7]:
# Check missing values
df_train.isnull().sum()

tweets        10
OFF_Labels     6
HS_Labels      6
dtype: int64

In [8]:
#drop any row have null values
df_train=df_train.dropna(how='any',axis=0)
df_valid=df_valid.dropna(how='any',axis=0)
# df_test.dropna()


In [9]:
df_train.isnull().sum()

tweets        0
OFF_Labels    0
HS_Labels     0
dtype: int64

In [10]:
x_train=df_train.tweets
x_valid=df_valid.tweets
x_test=df_test.tweets
y_train=df_train.HS_Labels
y_valid=df_valid.HS_Labels
y_test=df_test_labels.HS_Labels

pd.DataFrame(x_train)[:3]

,tweets
0,الحمدلله يارب فوز مهم يا زمالك.. كل الدعم ليكم يا رجالة ⚪🔴<LF>الدوري يا زمالك .. الدوري يا زمالك<LF>#صدارة_بس
1,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه
2,RT @USER: يا رب يا واحد يا أحد بحق يوم الاحد ان تهلك بني سعود المجرمين. لاجل اطفال اليمن شاركوا.


In [11]:
# x_valid.isnull().sum()

In [12]:
print('Labels on test: ',y_test.unique())
print('Labels on train: ',y_train.unique())

Labels on test:  ['HS' 'NOT_HS']
Labels on train:  ['NOT_HS' 'HS']


In [13]:
# y_test[:10]


In [14]:
#Check the shape
x_train.shape,y_train.shape,x_valid.shape,y_valid.shape,x_test.shape

((6984,), (6984,), (1000,), (1000,), (2000,))

In [15]:
#Label Encoding to convert labels to integer values
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
y_train=LE.fit_transform(y_train)
y_valid=LE.fit_transform(y_valid)
y_test=LE.fit_transform(y_test)
y_test[:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [16]:
# y_test = dict(zip(y_test, range(0,2)))
# y_train = dict(zip(y_train, range(0,2)))
# y_valid = dict(zip(y_valid, range(0,2)))
# y_test

In [17]:
# # Converting into list of tuple
# y_test = list(y_test.items())
# list = list(y_test)
# y_test = list(y_test.items())

# # Printing list of tuple
# print(y_test)
# example = list('easyhoss')  # here `list` refers to the builtin class
# list = list('abc')  # we create a variable `list` referencing an instance of `list`
# example = list('easyhoss')  # here `list` refers to the instance

In [18]:
# x_train = x_train.apply(lambda x: str(x[0], 'utf-8'))
# x_valid = x_valid.apply(lambda x:  str(x[0], 'utf-8'))
# x_test = x_test.apply(lambda x:  str(x[0], 'utf-8'))


In [19]:
# preprocessing function

text_cleaning_re = "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]|[.#،<>@,\\-_”“٪ًَ]"
def preprocess(text):
  # search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!','#']
  # replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ',' ']
  #remove tashkeel
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  text = re.sub(p_tashkeel,"", str(text))
  # text = text.replace('وو', 'و')
  # text = text.replace('يي', 'ي')
  # text = text.replace('اا', 'ا')
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  #remove longation
  text = re.sub(r'(.)\1+', r'\1\1', str(text)) 
  text = re.sub("[إأآا]", "ا", str(text))
  text = re.sub("ى", "ي", str(text))
  # text = re.sub("ؤ", "ء", text)
  # text = re.sub("ئ", "ء", text)
  text = re.sub("ة", "ه", str(text))
  # for i in range(0, len(search)):
  #       text = text.replace(search[i], replace[i])
  
   #trim    
  # text = text.strip()

  tokens = []
  for token in text.split():
    # if token not in stop_words:
      tokens.append(token)
  return " ".join(tokens)

In [20]:
# preprocess('مشكوووور hello 156562سٌلمُ@#')
preprocess('_ salam # @  حمقى 12 ممتازة ياااااااارب ؟ ! سٌلمُ')

'حمقي ممتازه ياارب ؟ ! سلم'

In [21]:
#Apply preproccing on Dataset
x_train = x_train.apply(lambda x: preprocess(x))
x_valid = x_valid.apply(lambda x: preprocess(x))
x_train[:10]

0                                                                            الحمدلله يارب فوز مهم يا زمالك كل الدعم ليكم يا رجاله ⚪🔴 الدوري يا زمالك الدوري يا زمالك صداره بس
1                                                                                                                                      فدوه يا بخت فدوه يا زمن واحد منكم يجيبه
2                                                                                       : يا رب يا واحد يا احد بحق يوم الاحد ان تهلك بني سعود المجرمين لاجل اطفال اليمن شاركوا
3                                                        : هوا الحريه يا وجع قلبي عليكي يا امي الله لا يحرق قلبك ويكسر بخاطرك للمره الف نقف اجلال لعظمه وقدره لين قلب الام ال…
4                                                                                                                                    يا بكون بحياتك الاهم يا اما ما بدي اكون 🎼
5                                                                                                    اخخ يا قلببي يا هالحلقه 

#Prepare data as model input.

In [22]:
x_train = x_train.astype(str)
x_valid = x_valid.astype(str)
x_test = x_test.astype(str)

##Tokenizer

In [23]:
#Define a tokenizer object
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#tokenize the text
train_encodings = tokenizer(list(x_train.values),
                            truncation=True, 
                            padding=True)
valid_encodings = tokenizer(list(x_valid.values),
                           truncation=True, 
                           padding=True)
test_encodings = tokenizer(list(x_test.values),
                           truncation=True, 
                           padding=True)

In [24]:
print(f'First sample: \' {x_train[:1]}\'')
print(f'train ids:   {train_encodings["input_ids"][0]}')
print(f'attention_mask:  {train_encodings["attention_mask"][0]}')


First sample: ' 0    الحمدلله يارب فوز مهم يا زمالك كل الدعم ليكم يا رجاله ⚪🔴 الدوري يا زمالك الدوري يا زمالك صداره بس
Name: tweets, dtype: object'
train ids:   [101, 24177, 66716, 10975, 19427, 54065, 29464, 472, 19927, 50444, 54065, 462, 61570, 12570, 16392, 24177, 60419, 89787, 59322, 54065, 79964, 10411, 100, 43208, 54065, 462, 61570, 12570, 43208, 54065, 462, 61570, 12570, 465, 22233, 10411, 452, 11206, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [25]:
type(valid_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),list(y_train)))
valid_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_encodings),
                                    list(y_valid)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                    list(y_test)))


#Fine-tuning with TensorFlow | Build model

In [27]:
# Define Collbacks
# from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
# my_callbacks = [
    # tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2),
    # tf.keras.callbacks.ReduceLROnPlateau(factor=0.1,
    #                                  min_lr = 0.01,
    #                                  monitor = 'val_loss',
    #                                  verbose = 1)
# ]

In [ ]:

# model = TFAutoModelForSequenceClassification.from_pretrained('monologg/biobert_v1.1_pubmed', from_pt=True)

model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_NAME,from_pt=True)
#chose the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=3e-5)
#define the loss function 
losss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    name='binary_crossentropy')
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
# train the model 
history = model.fit(train_dataset.shuffle(len(x_train)).batch(BATCH_SIZE),
          validation_data=valid_dataset.shuffle(len(x_valid)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          # callbacks=my_callbacks,
          batch_size=BATCH_SIZE
          # epochs=3, 
          # steps_per_epoch=115
          )


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

#Evaluation model

In [ ]:
import matplotlib.pyplot as plt
s, (at, al) = plt.subplots(2,1)
at.plot(history.history['accuracy'], c= 'b')
at.plot(history.history['val_accuracy'], c='r')
at.set_title('model accuracy')
at.set_ylabel('accuracy')
at.set_xlabel('epoch')
at.legend(['Model_train', 'Model_val'], loc='upper left')

al.plot(history.history['loss'], c='m')
al.plot(history.history['val_loss'], c='c')
al.set_title('model loss')
al.set_ylabel('loss')
al.set_xlabel('epoch')
al.legend(['train', 'val'], loc = 'upper left')

In [ ]:
# model evaluation on the test set
model.evaluate(test_dataset.shuffle(len(x_test)).batch(BATCH_SIZE), 
               return_dict=True, 
               batch_size=BATCH_SIZE)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# pred= compute_metrics(x_test.to_list())

In [ ]:
# model evaluation on the test set
# predict = model.predict(test_dataset.shuffle(len(x_test)).batch(BATCH_SIZE))
# predict

In [ ]:
# matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

#CM | 

In [ ]:
def predict_proba(text_list, model, tokenizer):  
    #tokenize the text
    encodings = tokenizer(text_list, 
                          max_length=120, 
                          truncation=True, 
                          padding=True)
    #transform to tf.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings)))
    #predict
    preds = model.predict(dataset.batch(1)).logits  
    
    #transform to array with probabilities
    res = tf.nn.sigmoid(preds).numpy()      
    # res = tf.nn.softmax(preds, axis=1).numpy()      

    return res

In [ ]:
pred= predict_proba(x_test.to_list(), model, tokenizer)

In [ ]:
pred= np.argmax(pred, axis=1)


In [ ]:
# pred=pred.argmax(-1)
# pred = np.array(pred)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))
print(accuracy_score(y_test,pred))

In [ ]:
  # preds = np.argmax(p.predictions, axis=1)
  # assert len(preds) == len(p.label_ids)
  # print(classification_report(p.label_ids,preds))
  # print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  # macro_f1 = f1_score(p.label_ids,preds,average='macro')
  # macro_precision = precision_score(p.label_ids,preds,average='macro')
  # macro_recall = recall_score(p.label_ids,preds,average='macro')
  # acc = accuracy_score(p.label_ids,preds)
  # return {
  #     'macro_f1' : macro_f1,
  #     'macro_f1_pos_neg' : macro_f1_pos_neg,  
  #     'macro_precision': macro_precision,
  #     'macro_recall': macro_recall,
  #     'accuracy': acc
  # }

In [ ]:
# strings_list =[""" هل يمكن للعرب الاتفاق على خطة مشتركة لاستقبال  اللاجئيين السوريين -كما يفعل غيرنا- حتى تنتهى المذابح ام ان هذا خارج إطار الإخوة العربية؟""",
#                """قذر اتفووو ماتيجى مصر وتورينا نفسك كدا ياجبان""",
#                """انت معلم السخرية الأول لم نتعلم منك شيء الا الاستهزاء والاستخفاف والاستظراف المتقن من خلال سكريبت انت محتاج علاج ابدا بنفسك""",
#                """ انت واشكالك اللي بتعبدو المرشد لكن احنا بنعبد ربنا بس ونحب اللي يحب بلدنا مش بنشتم بلدنا زيكو""",
#                "يا خنزير كفاك حقارة ",
#                " عظمة الخلق وكرمه ",
#                " دراسة تكشف عن تراجع #كراهية #المسلمين في #ليفربول",
#                "كل دعاية للحرب وكل #صراخ و #كراهية و كذب يصدر من أناس لا يذهبون #للحرب .!!",
#                "مرأة تعترف بقتل مسلم بدفعه نحو سكة قطار متحرك",
#                "الكورونا تليق بكم حسنًا اليد #طائفية #عنصرية #كراهية #ما_الها_حدود"
              
#                ]

In [ ]:
# predict_proba(strings_list, model, tokenizer)